In [ ]:
#This notebook contains functions necessary to implement Shamir's secret sharing
#Note that this code and the code that will be implemented into the project uses a linear representation instead of a finite field. 
#For any enterprise development the code must be changed to include a finite field build around a significantly large prime such as 2^127 -1 

In [15]:
#This cell contains necessary imports
import numpy as np
from scipy.interpolate import lagrange 
import pandas as pd
from random import seed
from random import randint
from numpy.polynomial.polynomial import Polynomial
import Crypto
#bytes object initialization 
from Crypto.PublicKey import RSA
#import bcrypt
import rsa

In [64]:
def main():
    S_list = create_secret_value_table()
    #generates a list of secret indicies
    RSA_list = generate_RSA_keypairs()
    #generates a dataframe of public private keypairs
    S_df = pd.DataFrame({"Secret ID": S_list})
    S_RSA_DF = pd.concat([S_df, RSA_list], axis = 1)
    S_RSA_DF.to_csv('RSA_Keys.csv')
    #concatonates the two frames creating a pair between secret ID and each set of keys
        
main()

In [37]:
#This cell creates a random set of 20 secret values for S, values will range between 100 and 1000
def create_secret_value_table():
    seed(42)
    S_list = []
    for i in range(20):
        S_list.append(randint(100,1000))
    S_list.sort()
    return S_list
    

In [47]:
#This cell creates a table of 20 key pairs to be combined with the secret list to act as a reference. NOTE SCRIPT REQUIRES 4 CORE CPU
def generate_RSA_keypairs():
    cols = ['public key', 'private key']
    N = 20
    RSA_list = pd.DataFrame(columns = cols)
    for i in range(N):
        (pubkey, privkey) = rsa.newkeys(4096, poolsize = 6)
        RSA_list = RSA_list.append({'public key' : pubkey , 'private key' : privkey}, ignore_index = True)
    return RSA_list

In [ ]:
#This function creates a master key for the secrets table. Deciding against second round of encryption for this piece
#def generate_encrypted_keys():
#    (master_pub, master_priv) = rsa.newkeys(4096, poolsize 6)
#    S_RSA_slice = S_RSA_DF.copy()
#    S_RSA_slice.columns= ['SecretID', 'PubKey', 'PrivKey']
#    S_RSA_slice['EncryptedPrivate'] = S_RSA_slice.apply(lambda row: rsa.encrypt(row.PrivKey, master_pub), axis = 1)
#    S_RSA_slice
 
    

In [73]:
#S_df = pd.DataFrame({"Secret ID": S_list})
#S_RSA_DF = pd.concat([S_df, RSA_list], axis = 1)

TypeError: ("object of type 'PrivateKey' has no len()", 'occurred at index 0')

In [ ]:
#This cell generates two values fC1 and fC2 using random int for the generation of the 2nd degree polynomial 
def generate_function_coefficients:
    return fC1, fC2

In [ ]:
#This cell generates the polynomial function for evaluation of shares, Shamir_Poly is a polynomial variable
def generate_polynomial_function:
    return Shamir_Poly

In [ ]:
#This cell calculates the shares off the Shamir_Poly. This function should take in the five agents that signed the transaction, calculate the shares for all five, then return their relevant value.
def generate_Shamir_Shares (holder1, holder2, holder3, holder4, holder5):
    return Share1, Share2, Share3, Share4, Share5